In [ ]:
import urllib.request
import time
from bs4 import BeautifulSoup
import os
import re
import html
import shutil

In [ ]:
def get_text(text):
    reg = re.compile('</h2>.*?<div class="comment">', flags=re.DOTALL)
    my_text = reg.findall(text)
    reg_tag = re.compile('<.*?>', re.DOTALL)
    reg_space = re.compile('\s{2,}', re.DOTALL)
    for t in my_text:
        clean_t = reg_space.sub("", t)
        clean_t = reg_tag.sub("", clean_t)
        clean_t = html.unescape(clean_t)
        return clean_t 

In [ ]:
 def get_meta(text):
    soup = BeautifulSoup(text, 'html.parser')
    article = soup.find('h2').get_text()
    date = soup.find('div', {'class': 'mndata'}).get_text()
    category = soup.find('h1').get_text()
    return article, date, category

In [ ]:
def my_path(date, counter, path_type):
    path = os.path.join('газета', path_type, date[6:10], date[3:5])
    if not os.path.exists(path):
        os.makedirs(path)
    path = os.path.join(path, str(counter))
    return path, counter

In [ ]:
def my_plain(counter, date, article, category, page_url, clean_t):
    path, counter = my_path(date, counter, 'plain')
    path += '.txt'
    with open(path, 'w', encoding='utf-8') as f:
        f.write('@au None\n@ti %s\n@da %s\n@topic %s\n@url %s\n%s' % (
                article, date, category, page_url, clean_t))
    return counter

In [ ]:
def mystem_text(counter, date):
    path, counter = my_path(date, counter, 'mystem-plain')
    path += '.txt'
    os.system('./mystem -id tmp.txt ' + path)
    path, counter = my_path(date, counter, 'mystem-xml')
    path += '.xml'
    os.system('./mystem -id tmp.txt ' + path)
    return counter

In [ ]:
def meta_data(counter, page_url, article, date, category):
    path, counter = my_path(date, counter, 'plain')
    row = '%s\t\t%s\t%s\tпублицистика\t%s\tнейтральный\tн-возраст\t' \
          'н-уровень\tрайонная\t%s\tСельская ' \
          'новь\t%s\tгазета\tРоссия\tПензенская область\tru '
    inf = row % (path + '.txt', article, date, category, page_url, date[6:10])
    return inf, counter

In [ ]:
def main():
    if os.path.exists('газета'):
        shutil.rmtree('газета')
    counter = 0
    os.makedirs('газета')
    for a in range(200, 3435):
        time.sleep(2)
        page_url = 'http://www.selsknov.ru/news-10-%d.html' % a
        try:
            page = urllib.request.urlopen(page_url)
            text = page.read().decode('cp1251')
        except urllib.error.HTTPError:
            print('Error at', page_url)
            continue
        clean_t = get_text(text)
        article, date, category = get_meta(text)
        if clean_t == '' or article == '':
            continue
        with open(os.path.join('газета', 'metadata.csv'), 'a',
                  encoding='utf-8') as csv:
            inf, counter = meta_data(counter, page_url, article,
                                     date, category)
            csv.write(inf + '\n')
            with open('tmp.txt', 'w', encoding='utf-8') as tmp:
                tmp.write(clean_t)
            counter = my_plain(counter, date, page_url,
                               article, category, clean_t)
            counter = mystem_text(counter, date)
            counter += 1
    os.remove('tmp.txt')

In [ ]:
if __name__== '__main__':
    main()
    print('done')